# 10 Fold CV

In [5]:
import pandas as pd
from Bio import SeqIO
import os
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax

from tensorflow.keras.regularizers import l1, l2


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler

import imblearn
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc
import os 
import pandas as pd
import numpy as np

In [17]:
os.chdir("/home/t326h379/OGP")


df_negative = pd.read_csv('Feature_Extraction_O_linked_Training_Negative_114307_Sites_less.txt',header=None)

df_positive = pd.read_csv('Feature_Extraction_O_linked_Training_Positive_4885_Sites_less.txt',header=None)

Header_name = ["Position","PID","Position_redundant","81 Window sequence","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive.columns = Header_name
df_negative.columns = Header_name


frames = [df_positive, df_negative]

O_linked_training = pd.concat(frames,ignore_index = True)

df_Train_array = O_linked_training.drop(["Position","PID","Position_redundant","81 Window sequence","S or T"],axis=1)
df_Train_array = np.array(df_Train_array)

X_train_full = df_Train_array

y_train_full = np.array([1]*4885+[0]*114144)

In [19]:
df_negative

,Position,PID,Position_redundant,81 Window sequence,S or T,1,2,3,4,5,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
0,92,Q9UBP0,92,YFSYPLFVGFALLRLVAFHLGLLFVWLCQRFSRALMAAKRSSGAAP...,S,0.090565,0.003615,0.265912,0.221255,0.272241,...,0.189914,0.008025,-0.074416,0.055804,-0.027644,-0.171877,0.242402,-0.013881,-0.022422,0.076613
1,616,A1L4H1,616,ISDPFSWSWIPGLGRDRDAWLPGELATKPSASVTASVLEKTTTKAP...,T,-0.399097,0.180620,-0.011469,0.183442,0.480593,...,-0.057433,-0.002528,0.333472,-0.083524,0.054328,-0.456104,-0.221937,0.233516,0.099400,-0.186329
2,447,Q4KMG0,447,IITAPVSAKVADGDFVTLSCNASGLPVPVIRWYDSHGLITSHPSQV...,S,0.224062,0.048616,0.022553,-0.029075,0.030947,...,0.013840,0.103943,0.244616,-0.073571,0.115669,0.126064,0.114876,-0.263312,0.215138,0.056720
3,234,P05121,234,FNGQWKTPFPDSSTHRRLFHKSDGSTVSVPMMAQTNKFNYTEFTTP...,T,0.119445,0.121850,0.369377,0.369897,0.012500,...,-0.115018,-0.155688,0.051944,0.058649,0.276462,-0.187079,-0.021619,0.214745,-0.043378,-0.120113
4,130,Q07960,130,ACRMPPSHQLDHSKLLGYLKHTLDQYVESDYTLLYLHHGLTSDNKP...,T,-0.029804,-0.012195,0.098315,-0.187430,0.174959,...,-0.062594,0.243982,-0.192061,0.074429,0.060254,-0.303652,0.218557,-0.179121,-0.222627,-0.108745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114139,954,Q8TEM1,954,LRIREGSGYFFLNTSTADVVKVAYQEARGVAMVHPLLPGSSTIMIH...,S,-0.373295,0.188129,-0.315040,-0.151053,-0.373331,...,0.046646,0.277940,-0.331345,-0.313045,0.199881,0.142525,0.323091,0.520124,-0.196773,0.170163
114140,45,P12109,45,RALLPLLLQACWTAAQDEPETPRAVAFQDCPVDLFFVLDTSESVAL...,S,0.274927,-0.068854,0.322756,0.071589,-0.260691,...,0.002543,0.350866,-0.266978,-0.229258,-0.064659,-0.413040,-0.067693,0.026467,0.048538,0.494500
114141,721,Q685J3,721,STYTEGSTPLTSMPVNTTLVASSEASTLSTTPVDTSTPVTTSTEAS...,T,-0.079311,0.448357,0.206784,0.247052,0.289342,...,0.146359,0.042260,0.229490,-0.253621,0.407581,-0.132414,-0.034729,-0.176205,0.063649,0.103462
114142,11864,Q8WXI7,11864,TLLVTGTSRVDLSPTASPGVSAKTAPLSTHPGTETSTMIPTSTLSL...,T,0.083592,-0.027967,0.200502,0.299424,0.469784,...,-0.243491,0.622060,0.429536,-0.161372,0.220750,-0.265639,-0.212433,-0.285282,0.537182,0.153588


In [20]:
df_positive

,Position,PID,Position_redundant,81 Window sequence,S or T,1,2,3,4,5,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
0,4060,Q8WXI7,4060,VSPYMDTSSTTQTSIISSPGSTAITKGPRTEITSSKRISSSFLAQS...,S,0.022986,-0.118263,0.177780,0.246446,-0.082226,...,-0.150221,0.583051,-0.020838,-0.194725,0.039820,-0.038010,0.268013,-0.160448,0.313330,0.031695
1,2733,Q02817,2733,TGTQTPTSTPISTTTTVTPTATPTGTQTPTLTPITTTTTVTSTPTP...,T,0.526688,0.228523,0.414098,0.151443,0.523440,...,0.490214,0.017934,0.195478,-0.074319,0.311159,-0.362215,-0.271415,0.040819,0.106888,0.164128
2,52,P15514,52,VLSLLILGSGHYAAGLDLNDTYSGKREPFSGDHSADGFEVTSRSEM...,T,-0.404962,0.329316,0.109950,-0.017265,0.073663,...,-0.018510,-0.008204,0.039515,0.071839,0.042044,-0.232947,-0.516805,-0.376891,0.200629,-0.278637
3,3737,Q02817,3737,TPTPTPTGTQTPTSTPITTTTTVTPTPTPTGTQTPTPTPISTTSTV...,S,0.115557,0.386895,0.535525,0.383236,-0.216051,...,0.198823,0.300182,0.126487,-0.285474,0.224651,-0.123275,0.157720,-0.332858,-0.044721,-0.287276
4,270,O00461,270,QVAEYKQLKDTLNRIPSLRKPDPAEQQNVTQVAHSPQGYNTAREKP...,T,0.103378,-0.081142,0.038349,-0.002521,-0.134780,...,0.003350,0.072490,0.102203,0.094363,0.133203,-0.011620,-0.009290,0.071256,-0.066625,0.169510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4880,292,Q14242,292,QTTPLAAMEALSTEPSATEALSMEPTTKRGLFIPFSVSSVTHKGIP...,T,-0.143447,0.091008,0.246226,-0.029177,0.333892,...,-0.279839,-0.087998,0.394984,0.096512,-0.036989,-0.536879,-0.548743,-0.279505,0.005524,-0.044220
4881,451,P15941,451,TAPPAHGVTSAPDTRPAPGSTAPPAHGVTSAPDTRPAPGSTAPPAH...,T,0.333800,0.024173,0.219554,0.103433,0.233328,...,-0.083353,0.139198,-0.114918,-0.232843,0.488561,-0.332134,-0.445608,0.203515,0.145742,0.257425
4882,2294,Q02817,2294,TPTGTQTPTSTPISTTTTVTPTPTPTGTQTPTPTPISTTTTVTPTP...,T,0.584030,0.198676,0.033160,0.123643,0.531183,...,0.478282,0.151042,0.143494,-0.107475,0.515316,-0.232197,-0.323919,-0.139580,0.341235,0.464046
4883,530,P02768,530,KTPVSDRVTKCCTESLVNRRPCFSALEVDETYVPKEFNAETFTFHA...,T,0.047491,0.125332,0.046872,0.058702,-0.168642,...,-0.178552,0.040738,-0.073922,0.169234,0.156390,-0.276549,-0.240751,0.020197,-0.074785,-0.145535


In [21]:
X_train_full.shape

(119029, 1024)

In [22]:
y_train_full.shape

(119029,)

# Do 10 fold Cross Validation for 2 Times

In [23]:
a = random.sample(range(1, 1000000), 1)

In [24]:
for i in a:
    seed = i
    print("Seed : ",seed)
    rus = RandomUnderSampler(random_state = seed)
    X_train, y_train = rus.fit_resample(X_train_full,y_train_full)

    X_train, y_train = shuffle(X_train,y_train)

    print(X_train_full.shape)
    print(y_train_full.shape)

    MCC = []
    kfold = StratifiedKFold(n_splits=10)
    cvscores, auc_scores, sn, sp, acc = list(), list(), list(), list(), list()

    for train_index, test_index in kfold.split(X_train, y_train):
        xtrain, xval = X_train[train_index], X_train[test_index]
        ytrain, yval = y_train[train_index], y_train[test_index]      

        X_train_10= xtrain
        Y_train_10=ytrain

        Y_train_10 = tf.keras.utils.to_categorical(Y_train_10,2)

        model = Sequential()

        model.add(Input(shape=(1024,)))

        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(32,activation='relu',name="Dense_1"))
        model.add(Dropout(0.3))

        model.add(Dense(2,activation='softmax',name="Dense_2"))


        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),loss="binary_crossentropy",metrics=["accuracy"])

        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                        monitor = 'val_accuracy',
                                        verbose=0, 
                                        save_weights_only=False,
                                        save_best_only=True)

        reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=3, verbose=1, min_delta=1e-4, mode='max')

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

        history = model.fit(X_train_10, Y_train_10,epochs=35,verbose=1,batch_size=256)

        pd.DataFrame(history.history).plot(figsize=(8,5))
        plt.grid(True)
        plt.gca().set_ylim(0,1)
        plt.show()
        plt.savefig('accuracy_loss@@@@@@@@@@@@@@@_curve.png', dpi=300, bbox_inches='tight')

        Y_pred = model.predict(xval)
        Y_pred = (Y_pred > 0.5)
        y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
        y_pred = np.array(y_pred)
        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",accuracy_score(yval, y_pred))

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(accuracy_score(yval, y_pred))

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   


        model.summary()




    print("Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("\n\n\n\n\n\n")

Seed :  284827
(119029, 1024)
(119029,)
Epoch 1/35
35/35 [==============================] - 0s 6ms/step - loss: 0.6934 - accuracy: 0.5289
Epoch 2/35
35/35 [==============================] - 0s 5ms/step - loss: 0.6672 - accuracy: 0.5648
Epoch 3/35
35/35 [==============================] - 0s 5ms/step - loss: 0.6491 - accuracy: 0.6016
Epoch 4/35
35/35 [==============================] - 0s 5ms/step - loss: 0.6309 - accuracy: 0.6368
Epoch 5/35
35/35 [==============================] - 0s 4ms/step - loss: 0.6183 - accuracy: 0.6593
Epoch 6/35
35/35 [==============================] - 0s 5ms/step - loss: 0.6075 - accuracy: 0.6775
Epoch 7/35
35/35 [==============================] - 0s 5ms/step - loss: 0.5982 - accuracy: 0.6901
Epoch 8/35
35/35 [==============================] - 0s 6ms/step - loss: 0.5890 - accuracy: 0.7050
Epoch 9/35
35/35 [==============================] - 0s 6ms/step - loss: 0.5794 - accuracy: 0.7136
Epoch 10/35
35/35 [==============================] - 0s 6ms/step - loss: 0.568

/opt/software/software/SciPy-bundle/2019.10-foss-2019b-Python-3.7.4/lib/python3.7/site-packages/pandas/plotting/_matplotlib/core.py:338: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


Matthews Correlation :  0.6510756164403059
Confusion Matrix : 
 [[418  71]
 [100 388]]
Accuracy on test set:    0.8249744114636642
Sensitivity:    0.7950819672131147 	 Specificity:    0.8548057259713702
Area Under Curve:    0.8249438465922424
Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_63 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_64 (Dropout)         (None, 256)               0         
_________________________________________________________________
Dense_1 (Dense)              (None, 32)                8224      
________

/opt/software/software/SciPy-bundle/2019.10-foss-2019b-Python-3.7.4/lib/python3.7/site-packages/pandas/plotting/_matplotlib/core.py:338: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


Matthews Correlation :  0.6271562161119743
Confusion Matrix : 
 [[416  73]
 [110 378]]
Accuracy on test set:    0.812691914022518
Sensitivity:    0.7745901639344263 	 Specificity:    0.8507157464212679
Area Under Curve:    0.8126529551778471
Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_66 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_67 (Dropout)         (None, 256)               0         
_________________________________________________________________
Dense_1 (Dense)              (None, 32)                8224      
_________

...

Matthews Correlation :  0.6011143974984375
Confusion Matrix : 
 [[398  90]
 [105 384]]
Accuracy on test set:    0.8004094165813715
Sensitivity:    0.7852760736196319 	 Specificity:    0.8155737704918032
Area Under Curve:    0.8004249220557175
Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_84 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_85 (Dropout)         (None, 256)               0         
_________________________________________________________________
Dense_1 (Dense)              (None, 32)                8224      
________

/opt/software/software/SciPy-bundle/2019.10-foss-2019b-Python-3.7.4/lib/python3.7/site-packages/pandas/plotting/_matplotlib/core.py:338: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


Matthews Correlation :  0.5955432781312355
Confusion Matrix : 
 [[402  86]
 [112 377]]
Accuracy on test set:    0.797338792221085
Sensitivity:    0.7709611451942741 	 Specificity:    0.8237704918032787
Area Under Curve:    0.7973658184987764
Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_87 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_88 (Dropout)         (None, 256)               0         
_________________________________________________________________
Dense_1 (Dense)              (None, 32)                8224      
_________

/opt/software/software/SciPy-bundle/2019.10-foss-2019b-Python-3.7.4/lib/python3.7/site-packages/pandas/plotting/_matplotlib/core.py:338: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


Matthews Correlation :  0.6453241736438683
Confusion Matrix : 
 [[426  62]
 [113 376]]
Accuracy on test set:    0.8208802456499488
Sensitivity:    0.7689161554192229 	 Specificity:    0.8729508196721312
Area Under Curve:    0.820933487545677
Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_90 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_61 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_91 (Dropout)         (None, 256)               0         
_________________________________________________________________
Dense_1 (Dense)              (None, 32)                8224      
_________

# Independent Testing

In [27]:
import os 
import pandas as pd
import numpy as np

os.chdir("/home/t326h379/OGP")
import pandas as pd
from Bio import SeqIO
import os
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax

from tensorflow.keras.regularizers import l1, l2


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import imblearn
a = random.sample(range(1, 1000000), 10)

# Training Dataset

df_negative = pd.read_csv('Feature_Extraction_O_linked_Training_Negative_114307_Sites_less.txt',header=None)

df_positive = pd.read_csv('Feature_Extraction_O_linked_Training_Positive_4885_Sites_less.txt',header=None)

Header_name = ["Position","PID","Position_redundant","81 Window sequence","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive.columns = Header_name
df_negative.columns = Header_name


frames = [df_positive, df_negative]

O_linked_training = pd.concat(frames,ignore_index = True)

df_Train_array = O_linked_training.drop(["Position","PID","Position_redundant","81 Window sequence","S or T"],axis=1)
df_Train_array = np.array(df_Train_array)

X_train_full = df_Train_array

y_train_full = np.array([1]*4885+[0]*114144)

# Independent Test Dataset
df_negative_test = pd.read_csv('Feature_Extraction_O_linked_Testing_Negative_11466_Sites_less.txt',header=None)

df_positive_test = pd.read_csv('Feature_Extraction_O_linked_Testing_Positive_375_Sites_less.txt',header=None)

Header_name = ["Position","PID","Position_redundant","81 Window sequence","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive_test.columns = Header_name

df_negative_test.columns = Header_name


frames_test = [df_positive_test, df_negative_test]

O_linked_testing = pd.concat(frames_test,ignore_index = True)

df_Test_array = O_linked_testing.drop(["Position","PID","Position_redundant","81 Window sequence","S or T"],axis=1)
df_Test_array = np.array(df_Test_array)

X_test_full = df_Test_array

y_test_full = np.array([1]*374+[0]*11466)

# Training Starts From Here

from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

from imblearn.under_sampling import RandomUnderSampler

# Run the model for Three times and choose the best answer among them
a = random.sample(range(1, 1000000), 3)

for i in a:
    seed = i
    print("Seed : ", seed)

    rus = RandomUnderSampler(random_state = seed)
    X_train, y_train = rus.fit_resample(X_train_full,y_train_full)

    x_train, x_val, y_train_1, y_val = train_test_split(X_train, y_train,random_state =21, test_size=0.1)

    y_train_1 = tf.keras.utils.to_categorical(y_train_1,2)
    y_val = tf.keras.utils.to_categorical(y_val,2)


    model = Sequential()

    model.add(Input(shape=(1024,)))

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(32,activation='relu',name="Dense_1"))
    model.add(Dropout(0.3))

    model.add(Dense(2,activation='softmax',name="Dense_2"))


    model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])

    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                    monitor = 'val_accuracy',
                                    verbose=0, 
                                    save_weights_only=False,
                                    save_best_only=True)

    reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=5, verbose=1, min_delta=1e-4, mode='max')

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

    history = model.fit(x_train, y_train_1,epochs=400,verbose=1,batch_size=256,
                            callbacks=[checkpointer,reduce_lr_acc, early_stopping],validation_data=(x_val, y_val))
    
    from imblearn.under_sampling import RandomUnderSampler

    rus = RandomUnderSampler(random_state = seed)
    X_independent, y_independent = rus.fit_resample(X_test_full,y_test_full)
    
    Y_pred = model.predict(X_independent)
    Y_pred = (Y_pred > 0.5)
    y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
    y_pred = np.array(y_pred)

    confusion = confusion_matrix(y_independent,y_pred)

    print("Matthews Correlation : ",matthews_corrcoef(y_independent, y_pred))
    print("Confusion Matrix : \n",confusion_matrix(y_independent, y_pred))
    print("Accuracy on test set:   ",accuracy_score(y_independent, y_pred))

    cm = confusion_matrix(y_independent, y_pred)

    TP = cm[1][1]
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]

    mcc = matthews_corrcoef(y_independent, y_pred)

    Sensitivity = TP/(TP+FN)

    Specificity = TN/(TN+FP)

    print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)

    print(classification_report(y_independent, y_pred))

    fpr, tpr, _ = roc_curve(y_independent, y_pred)

    roc_auc_test = auc(fpr,tpr)



    print("Area Under Curve:   ",roc_auc_test)

Seed :  882022
Epoch 1/400
35/35 [==============================] - 1s 24ms/step - loss: 0.5057 - accuracy: 0.7649 - val_loss: 0.4402 - val_accuracy: 0.7932
Epoch 2/400
35/35 [==============================] - 1s 16ms/step - loss: 0.4067 - accuracy: 0.8310 - val_loss: 0.4038 - val_accuracy: 0.8025
Epoch 3/400
35/35 [==============================] - 5s 147ms/step - loss: 0.3549 - accuracy: 0.8607 - val_loss: 0.3949 - val_accuracy: 0.8229
Epoch 4/400
35/35 [==============================] - 1s 20ms/step - loss: 0.2980 - accuracy: 0.8826 - val_loss: 0.3884 - val_accuracy: 0.8403
Epoch 5/400
35/35 [==============================] - 0s 7ms/step - loss: 0.2555 - accuracy: 0.9041 - val_loss: 0.4303 - val_accuracy: 0.8291
Epoch 6/400
35/35 [==============================] - 0s 7ms/step - loss: 0.2139 - accuracy: 0.9185 - val_loss: 0.4488 - val_accuracy: 0.8332
Epoch 7/400
35/35 [==============================] - 0s 6ms/step - loss: 0.1801 - accuracy: 0.9326 - val_loss: 0.5432 - val_accuracy: 

# Thank You!